In [ ]:
import torch
from FasterRCNN_crowdhuman import train_fasterrcnn_crowdhuman
import cv2

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model, log_dir, history = train_fasterrcnn_crowdhuman(
    num_epochs=5,
    batch_size=2,
    lr=0.005,
    num_workers=0,
    device=device,
    run_name=None,
    score_thresh=0.5,
    iou_thresh=0.5,
)
print("训练日志目录:", log_dir)
print("best model 路径:", history["best_model"])

In [ ]:
from FasterRCNN_crowdhuman import evaluate_best_model_on_val

device = "cuda:0" if torch.cuda.is_available() else "cpu"

metrics, vis_dir, metrics_txt = evaluate_best_model_on_val(
    log_dir=log_dir,
    device=device,
    batch_size=4,
    num_workers=4,
    score_thresh=0.5,
    iou_thresh=0.5,
    save_subdir="val_best",
)

print("验证集指标:", metrics)
print("可视化结果目录:", vis_dir)
print("指标 txt 路径:", metrics_txt)


In [ ]:
import torch
from pathlib import Path
from FasterRCNN_crowdhuman import (
    create_fasterrcnn_resnet50_fpn_v2,
    detect_people_in_image,
    DATA_ROOT,
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# 1. 加载 best model
best_model_path = history["best_model"]
print("加载模型:", best_model_path)

NUM_CLASSES = 2  # [background, person]
model = create_fasterrcnn_resnet50_fpn_v2(num_classes=NUM_CLASSES, pretrained=False)
state_dict = torch.load(best_model_path, map_location=device)
model.load_state_dict(state_dict)
model.to(device)

# 2. 随便选一张 val 图
val_img_dir = DATA_ROOT / "images" / "val"
test_img_path = next(val_img_dir.glob("*.jpg"))  # 拿第一张
print("测试图像:", test_img_path)

# 3. 检测 + 可视化
save_vis_path = Path(log_dir) / "single_test" / test_img_path.name

people_count, vis_img = detect_people_in_image(
    model,
    device=device,
    image_path=str(test_img_path),
    score_thresh=0.5,
    save_path=str(save_vis_path),
)

print("检测到人数:", people_count)
from matplotlib import pyplot as plt

plt.imshow(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()
